# Notebook para el proyecto de predicción de atrasos de vuelos - HACKATHON ONE

## Data Engineer (DE)

Sección para las tareas de Data Engineer.

Encargado: Ismael Cerda

### Selección y Limpieza de Datos

Base de datos obtenida de: https://www.kaggle.com/datasets/jimschacko/airlines-dataset-to-predict-a-delay?select=Airlines.csv

In [18]:
import pandas as pd

url="https://raw.githubusercontent.com/degartHub/nocountry-h12-25-equipo27-datascience/refs/heads/main/data/Airlines.csv"
df = pd.read_csv(url)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539383 entries, 0 to 539382
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           539383 non-null  int64 
 1   Airline      539383 non-null  object
 2   Flight       539383 non-null  int64 
 3   AirportFrom  539383 non-null  object
 4   AirportTo    539383 non-null  object
 5   DayOfWeek    539383 non-null  int64 
 6   Time         539383 non-null  int64 
 7   Length       539383 non-null  int64 
 8   Delay        539383 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 37.0+ MB


In [20]:
df["Time"].agg(["min", "max"])

,Time
min,10
max,1439


La base de datos cuenta con un total de 539.383 registros y un total de 9 columnas, siendo estas:

- <u>**id**</u>= Identifica la fila del registro.

- <u>**Airline**</u>= Aerolínea.

- <u>**Flight**</u>= Número de la aeronave.

- <u>**Airport From**</u>= Aeropuerto de salida.

- <u>**Airport To**</u>= Aeropuerto de destino.

- <u>**DayOfWeek**</u>= Día de la semana (en números).

- <u>**Time**</u>= Hora de salida medida en minutos a partir de la medianoche (rango de [10,1439], lo que podría ser el equivalente a un día).

- <u>**Lenght**</u>= Duración del vuelo en minutos.

- <u>**Delay**</u>= Con retraso (1), sin retraso (0).

In [21]:
df.sample(n=5)

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
533473,533474,DL,1452,ATL,CVG,5,852,88,0
527281,527282,WN,2178,BWI,BOS,5,515,85,0
98061,98062,WN,723,BWI,RDU,1,1000,65,1
186226,186227,US,456,PHX,SLC,6,1078,97,0
144727,144728,OO,6377,DEN,CPR,4,684,60,0


Las colummnas a eliminar serán:
- ID: Es un identificador para la tabla en sí
- Flight: Identifica el número de avión, no es relevante.

In [22]:
df = df.drop(columns=["id", "Flight"])

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539383 entries, 0 to 539382
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Airline      539383 non-null  object
 1   AirportFrom  539383 non-null  object
 2   AirportTo    539383 non-null  object
 3   DayOfWeek    539383 non-null  int64 
 4   Time         539383 non-null  int64 
 5   Length       539383 non-null  int64 
 6   Delay        539383 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 28.8+ MB


## Feature Architect (FA)

Sección para las tareas de Feature Architect

Encargado: Eduardo Ayala

### Ingeniería de Atributos

**Acerca de los datos faltantes**

Nos faltan fechas y distancia recorrida en km

*   Si observamos los valores mínimos y máximos de la columna `Time` se ve que están en el rango (10, 1439), que corresponde aproximadamente a los minutos que tiene un día completo, y, aparte, hay una columna `Lenght` con el tiempo de vuelo. Como son ~540 mil vuelos no se puede asumir que son todos del mismo día, es algo más parecido a los vuelos de 1 mes en USA. Así que se crearán las fechas de partida de manera sintética para 1 mes.
*   Adicionalmente a lo anterior, nos piden la distancia en kilómetros, pero tenemos el tiempo de vuelo; así que haremos la conversión estimando $800\frac{km}{\text{hr}}$ (velocidad promedio de un vuelo comercial).

Sumado a lo anterior, vamos a renombrar las columnas de acuerdo al contrato con backend.

**Acerca de la eficiencia y reducción de memoria**

A continuación se describen las optimizaciones aplicadas enfocadas en reducir el uso de memoria y mantener consistencia con backend.

1. Transformación de fechas a formato datetime

   - Las columnas de fecha se almacenaron como `datetime64[ns]` en lugar de strings.
   - La columna **`fecha_partida`** se generó en formato **ISO-8601** (`YYYY-MM-DDTHH:MM:SS`), compatible con backend.

 2. Creación de columnas de hora y día de la semana
    - Las variables temporales (`hora_salida`, `hora_llegada`, `dia_semana`) se extrajeron desde columnas datetime.
    - Se guardaron como **int8**: ocupan 1 byte por celda y tienen rango acotado (horas y días), sin pérdida de información.

3. One-Hot-Encoding de variables categóricas

    - Se utilizó `OneHotEncoder(handle_unknown='ignore')` para evitar errores en producción.
    - El resultado del encoding se convirtió a **uint8**.

4. Escalado de la variable de distancia (cuando el modelo lo requiere)

    - La variable **`distancia_km`** se convirtió a **float32**.
    - Para modelos que lo requieren (ej. Regresión Logística). Se usó `StandardScaler(with_mean=False)`, compatible con matrices sparse, evitando densificar el dataset.

In [24]:
import numpy as np
import pandas as pd

np.random.seed(42)
VELOCIDAD_PROMEDIO_KMH = 800

# Renombre según contrato backend
df = df.rename(columns={
    'Airline': 'aerolinea',
    'AirportFrom': 'origen',
    'AirportTo': 'destino',
    'Length': 'duration_min',
    'Delay': 'retraso'
})

# Distancia en km
df['distancia_km'] = (df['duration_min'] / 60) * VELOCIDAD_PROMEDIO_KMH

# Fechas base
start_date = pd.to_datetime('2018-12-01')
end_date = pd.to_datetime('2018-12-31')

random_days = np.random.randint(
    0, (end_date - start_date).days + 1, size=len(df)
)

df['FlightDate'] = (
    start_date + pd.to_timedelta(random_days, unit='D')
).normalize()

# Datetime salida y llegada
df['DepartureDateTime'] = (
    df['FlightDate'] + pd.to_timedelta(df['Time'], unit='m')
)

df['ArrivalDateTime'] = (
    df['DepartureDateTime'] + pd.to_timedelta(df['duration_min'], unit='m')
)

In [25]:
df['hora_salida'] = df['DepartureDateTime'].dt.hour.astype('int8')
df['hora_llegada'] = df['ArrivalDateTime'].dt.hour.astype('int8')
df['dia_semana'] = df['DepartureDateTime'].dt.dayofweek.astype('int8')

# Fecha de partida en formato backend (ISO-8601)
df['fecha_partida'] = df['DepartureDateTime'].dt.strftime('%Y-%m-%dT%H:%M:%S')

In [26]:
from sklearn.preprocessing import OneHotEncoder

categorical_features = ['aerolinea', 'origen', 'destino']

ohe = OneHotEncoder(
    sparse_output=False,
    handle_unknown='ignore'
)

X_cat = ohe.fit_transform(df[categorical_features])

X_cat = pd.DataFrame(
    X_cat,
    columns=ohe.get_feature_names_out(categorical_features),
    index=df.index
)

In [27]:
from sklearn.preprocessing import StandardScaler
from scipy import sparse

numeric_features = [
    'distancia_km',
    'hora_salida',
    'hora_llegada',
    'dia_semana'
]

X_num = df[numeric_features]

X_logreg = pd.concat([X_num, X_cat], axis=1)
X_rf = pd.concat([X_num, X_cat], axis=1)

y = df['retraso']

# Reducción de memoria
X_logreg[numeric_features] = X_logreg[numeric_features].astype('float32')
X_rf[numeric_features] = X_rf[numeric_features].astype('float32')

X_logreg[X_cat.columns] = X_cat.astype('uint8')
X_rf[X_cat.columns] = X_cat.astype('uint8')

# Sparse + escalado (solo Logistic Regression)
X_logreg_sparse = sparse.csr_matrix(X_logreg.values)

scaler = StandardScaler(with_mean=False)
num_idx = [X_logreg.columns.get_loc(c) for c in numeric_features]

X_logreg_scaled = X_logreg_sparse.copy()
X_logreg_scaled[:, num_idx] = scaler.fit_transform(
    X_logreg_sparse[:, num_idx]
)

In [28]:
import joblib

# Guardar transformadores
joblib.dump(ohe, 'onehot_encoder.pkl')
joblib.dump(scaler, 'scaler_logreg.pkl')

print("Transformadores guardados para producción")

# Mostrar las columnas para Backend en el formato correcto
print(
    df[
        ['aerolinea', 'origen', 'destino', 'fecha_partida', 'distancia_km']
    ].head()
)

Transformadores guardados para producción
  aerolinea origen destino        fecha_partida  distancia_km
0        CO    SFO     IAH  2018-12-07T00:15:00   2733.333333
1        US    PHX     CLT  2018-12-20T00:15:00   2960.000000
2        AA    LAX     DFW  2018-12-29T00:20:00   2200.000000
3        AA    SFO     DFW  2018-12-15T00:20:00   2600.000000
4        AS    ANC     SEA  2018-12-11T00:30:00   2693.333333


## Machine Learning Engineer (MLE)

Sección para las tareas de Machine Learning Engineer

Encargado: Luis Jácome

### Entrenamiento y Evaluación base

##Split de Datos (Train/Test) con un random_state fijo
En esta primera tarea separaremos los datos en entrenamiento y prueba con el objetivo de que sea reproducible, se encuentre balanceada y lista para entrenar al modelo al cual se le asignará el nombre de champion.

In [29]:
from sklearn.model_selection import train_test_split

# --------------------------------------------------
# SPLIT TRAIN / TEST (MLE - Tarea 1)
# --------------------------------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X_logreg_scaled,
    y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

# Verificación rápida
print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

print("\nDistribución Delay (train):")
print(y_train.value_counts(normalize=True))

print("\nDistribución Delay (test):")
print(y_test.value_counts(normalize=True))


Train shape: (431506, 608)
Test shape: (107877, 608)

Distribución Delay (train):
retraso
0    0.554558
1    0.445442
Name: proportion, dtype: float64

Distribución Delay (test):
retraso
0    0.554558
1    0.445442
Name: proportion, dtype: float64


## Entrenar el modelo seleccionado con parametros por defecto
Entrenaremos un modelo baseline usando los datos ya escalados y dejar el modelo listo para inferencia.
Se utilizará Logistic Regression para el entrenamiento ya que es un modelo interpretable, rápido, robusto y adecuado como baselinepara clasificación binaria.

In [30]:
from sklearn.linear_model import LogisticRegression
import joblib

# --------------------------------------------------
# ENTRENAMIENTO MODELO BASE - LOGISTIC REGRESSION
# --------------------------------------------------

champion = LogisticRegression(
    random_state=42,
    max_iter=1000,
    solver='liblinear'
)

champion.fit(X_train, y_train)

print("Modelo Logistic Regression entrenado correctamente")

# --------------------------------------------------
# GUARDAR MODELO PARA PRODUCCIÓN
# --------------------------------------------------

joblib.dump(champion, 'champion.pkl')

print("Modelo guardado como champion.pkl")


Modelo Logistic Regression entrenado correctamente
Modelo guardado como champion.pkl


## Evaluación del modelo
evaluaremos el modelo baseline mediante métricas estándar de clasificación binaria (Accuracy, Precision, Recall y F1-score) tanto en el conjunto de entrenamiento como de prueba, utilizando un umbral de decisión por defecto de 0.5

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# --------------------------------------------------
# PREDICCIONES
# --------------------------------------------------
y_train_pred = champion.predict(X_train)
y_test_pred = champion.predict(X_test)
# --------------------------------------------------
# MÉTRICAS TRAIN
# --------------------------------------------------
train_metrics = {
    "Accuracy": accuracy_score(y_train, y_train_pred),
    "Precision": precision_score(y_train, y_train_pred),
    "Recall": recall_score(y_train, y_train_pred),
    "F1-Score": f1_score(y_train, y_train_pred)
}
# --------------------------------------------------
# MÉTRICAS TEST
# --------------------------------------------------
test_metrics = {
    "Accuracy": accuracy_score(y_test, y_test_pred),
    "Precision": precision_score(y_test, y_test_pred),
    "Recall": recall_score(y_test, y_test_pred),
    "F1-Score": f1_score(y_test, y_test_pred)
}
# --------------------------------------------------
# MOSTRAR RESULTADOS
# --------------------------------------------------
print("Métricas Train:")
for k, v in train_metrics.items():
    print(f"{k}: {v:.4f}")

print("\nMétricas Test:")
for k, v in test_metrics.items():
    print(f"{k}: {v:.4f}")

Métricas Train:
Accuracy: 0.6452
Precision: 0.6318
Recall: 0.4879
F1-Score: 0.5506

Métricas Test:
Accuracy: 0.6459
Precision: 0.6328
Recall: 0.4884
F1-Score: 0.5513


## Validar que el modelo no tenga overfitting excesivo
La ausencia de overfitting se evidencia en la similitud casi exacta entre las métricas de entrenamiento y prueba. Las diferencias inferiores al 0.1% indican que el modelo generaliza correctamente y no presenta alta varianza.

## Seleccionar el umbral de probabilidad óptimo para la clasificación


In [32]:
# Probabilidad de clase positiva (Delay = 1)
y_proba_test = champion.predict_proba(X_test)[:, 1]

In [33]:
# Rango de umbrales a evaluar
thresholds = np.arange(0.1, 0.9, 0.05)

results = []

for threshold in thresholds:
    y_pred_threshold = (y_proba_test >= threshold).astype(int)

    precision = precision_score(y_test, y_pred_threshold)
    recall = recall_score(y_test, y_pred_threshold)
    f1 = f1_score(y_test, y_pred_threshold)

    results.append({
        "Threshold": threshold,
        "Precision": precision,
        "Recall": recall,
        "F1_score": f1
    })

# Resultados en DataFrame
threshold_df = pd.DataFrame(results)

threshold_df


,Threshold,Precision,Recall,F1_score
0,0.10,0.445741,0.999854,0.616599
1,0.15,0.449305,0.996899,0.619431
2,0.20,0.459582,0.981416,0.626012
3,0.25,0.477434,0.945102,0.634394
4,0.30,0.502348,0.883670,0.640555
5,0.35,0.530659,0.798181,0.637492
6,0.40,0.562636,0.697626,0.622901
7,0.45,0.596231,0.591305,0.593758
8,0.50,0.632785,0.488419,0.551308
9,0.55,0.673402,0.396687,0.499267


In [34]:
# selección del umbral óptimo (máximo F1)
best_threshold_row = threshold_df.loc[threshold_df['F1_score'].idxmax()]
best_threshold_row

,4
Threshold,0.300000
Precision,0.502348
Recall,0.883670
F1_score,0.640555


In [35]:
# guardado del umbral óptimo
best_threshold = best_threshold_row['Threshold']

print(f"Umbral óptimo seleccionado: {best_threshold:.2f}")


Umbral óptimo seleccionado: 0.30


In [36]:
# Comparación directa vs umbral 0.5
# Predicción con umbral por defecto
y_pred_default = (y_proba_test >= 0.5).astype(int)

# Predicción con umbral óptimo
y_pred_optimal = (y_proba_test >= best_threshold).astype(int)

comparison = pd.DataFrame({
    "Metric": ["Precision", "Recall", "F1_score"],
    "Threshold_0.5": [
        precision_score(y_test, y_pred_default),
        recall_score(y_test, y_pred_default),
        f1_score(y_test, y_pred_default)
    ],
    "Optimal_Threshold": [
        precision_score(y_test, y_pred_optimal),
        recall_score(y_test, y_pred_optimal),
        f1_score(y_test, y_pred_optimal)
    ]
})

comparison


,Metric,Threshold_0.5,Optimal_Threshold
0,Precision,0.632785,0.502348
1,Recall,0.488419,0.883670
2,F1_score,0.551308,0.640555


Al evaluar distintos umbrales de decisión, se identificó un umbral óptimo que maximiza el F1-score el cual es 0.3. Este ajuste incrementa significativamente la capacidad del modelo para detectar vuelos retrasados (Recall ≈ 88%), a costa de una reducción moderada en Precision, logrando un mejor equilibrio general sin necesidad de reentrenar el modelo.

## Machine Learning Operations (MLOps)

Sección para las tareas de Machine Learning Operations

Encargado: Nicolás Staffelbach

### Microservicio Python

#### Validación de las versiones de las librerias en Colab


```
!pip show fastapi...
```
Este código tiene como fin el saber las versiones de las librerias utilizadas en el entorno Google Colab, para la creación del archivo `requirements.txt` para garantizar el funcionamiento del modelo en producción.


In [37]:
#!pip show fastapi scikit-learn pandas numpy joblib uvicorn pydantic

#### Script de carga del modelo

En esta sección se desarrolla la creación del pipeline de carga del encoder, scaler y modelo, para su posterior uso en producción, garantizando el uso de los mismos objetos utilizados para el entrenamiento del modelo. Y también garantizando la optimización de la API.

In [39]:
# ============================================================
# MLOps – Build Inference Pipeline (REPRODUCIBLE)
# ============================================================

import joblib
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# ============================================================
# 1. Cargar artefactos del notebook
# ============================================================

encoder = joblib.load("onehot_encoder.pkl")      # FA
scaler = joblib.load("scaler_logreg.pkl")        # FA
model = joblib.load("champion.pkl")              # MLE

print("Encoder, Scaler y Modelo cargados correctamente")

# ============================================================
# 2. Columnas FINALES del modelo
# ============================================================

# Derivadas del contrato API
categorical_features = [
    "aerolinea",
    "origen",
    "destino",
    "dia_semana"      # derivada de fecha_partida
]

numerical_features = [
    "hora_salida",    # derivada de fecha_partida
    "distancia_km"
]

# ============================================================
# 3. Preprocesamiento
# ============================================================

preprocessor = ColumnTransformer(
    transformers=[
        ("categorical", encoder, categorical_features),
        ("numerical", scaler, numerical_features)
    ],
    remainder="drop"
)

# ============================================================
# 4. Pipeline de INFERENCIA (sin fit)
# ============================================================

pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", model)
    ]
)

print("Pipeline de inferencia construido correctamente")

# ============================================================
# 5. Exportar ARTEFACTO FINAL
# ============================================================

joblib.dump(pipeline, "flight_delay_inference_pipeline.joblib")

print("Pipeline exportado como flight_delay_inference_pipeline.joblib")


Encoder, Scaler y Modelo cargados correctamente
Pipeline de inferencia construido correctamente
Pipeline exportado como flight_delay_inference_pipeline.joblib


#### Creación del microservicio

A través del Notebook se crea un archivo `app.py` para la implementación del microservicio a través de la libreria `fastapi`

In [40]:
# ============================================================
# 1. Crear el archivo para el microservicio
# ============================================================
%%writefile app.py

# ============================================================
# 2. Importación de las librerias necesarias
# ============================================================

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

import joblib
import pandas as pd
from datetime import datetime
import time

# ============================================================
# 3. METADATOS de la app
# ============================================================

app = FastAPI(
    title="Flight Delay Predictor API",
    version="0.1", #Versión de prueba para BackEnd y MVP
    description="Microservicio de predicción de retrasos de vuelos"
)

# ============================================================
# 4. Carga del PIPELINE
# ============================================================

try:
    pipeline = joblib.load("flight_delay_inference_pipeline.joblib")
except Exception as e:
    raise RuntimeError(f"Error cargando el pipeline: {e}")

# ============================================================
# 5. Definición del contrato en la API
# ============================================================

class FlightRequest(BaseModel):
    aerolinea: str
    origen: str
    destino: str
    fecha_partida: str
    distancia_km: float

# ============================================================
# 6. Función de transformación de datos para el Modelo
# ============================================================

def parse_request_to_dataframe(request: FlightRequest) -> pd.DataFrame:
    try:
        dt = datetime.fromisoformat(request.fecha_partida)
    except ValueError:
        raise HTTPException(
            status_code=400,
            detail="fecha_partida debe estar en formato ISO: YYYY-MM-DDTHH:MM:SS"
        )

    data = {
        "aerolinea": request.aerolinea,
        "origen": request.origen,
        "destino": request.destino,
        "dia_semana": dt.strftime("%A"),
        "hora_salida": dt.hour,
        "distancia_km": request.distancia_km
    }

    return pd.DataFrame([data])

# ============================================================
# 7. Definición del ENDPOINT /predict
# ============================================================

@app.post("/predict")
def predict_flight_delay(request: FlightRequest):
    start_time = time.time()

    df = parse_request_to_dataframe(request)

    proba = pipeline.predict_proba(df)[0][1]
    prediction = "Retrasado" if proba >= 0.5 else "Puntual"

    latency_ms = (time.time() - start_time) * 1000

    return {
        "prevision": prediction,
        "probabilidad": round(proba, 3),
        "latencia_ms": round(latency_ms, 2)
    }

Writing app.py


## Data Analyst (DA)

Sección para las tareas de Data Analyst

Encargado: David Aragón

### Análisis de Datos Exploratorio EDA